In [45]:
import os
import pandas as pd

file_list = [file for file in os.scandir('./') if file.name.split('.')[-1] == 'xlsx']

In [46]:
dataframes = []

for file in file_list:
    df = pd.read_excel(file)
    dataframes.append(df)

In [47]:
full_df = pd.concat(dataframes)

In [49]:
import geopandas as gpd

geo_comunas = gpd.read_file('../datasets/division_comunal/', crs={'init': 'epsg:5361'})
del geo_comunas['SHAPE_Le_1']

DriverError: ../datasets/division_comunal/: No such file or directory

In [50]:
comunas_lnc = [comuna.lower() for comuna in geo_comunas.NOM_COM.unique()]

for comuna_servel in  full_df.Comuna.unique():
    if not comuna_servel.lower() in comunas_lnc:
        print(f'nos falta {comuna_servel.lower()}')


NameError: name 'geo_comunas' is not defined

In [51]:
full_df.columns

Index(['Tipo de Elección', 'Cargo', 'Fecha de Elección', 'Año de Elección',
       'Inicio de Período', 'Fin de Período', 'Período', 'Id Región', 'Región',
       'Circunscripción Senatorial', 'Distrito', 'Comuna',
       'Circunscripción Electoral', 'Nombre Circunscripción Electoral',
       'Circunscripción Provincial', 'Candidato (a)', 'Nombres',
       'Apellido Paterno', 'Apellido Materno', 'Sexo', 'Electo(a)', 'Partido',
       'Sigla Partido', 'Lista', 'Votos Totales', 'Nominado(a)', 'Sexo Mesa',
       'Electo', 'Unnamed: 0', 'nombre', 'votos', 'porcentaje', 'cargo',
       'Votación Presidencial', 'Provincia', 'Nombre Provincia', 'Continente',
       'Pais'],
      dtype='object')

In [ ]:
full_df.Sexo.unique()

In [ ]:
full_df.query('Sexo == "MUJER"').query('`Sexo Mesa` == "MUJER"').query('Comuna == "LAS CONDES"').query('Nombres == "EVA"').to_csv('sample.csv')

In [ ]:
data = []

votos_mujeres = full_df.query('`Sexo Mesa` == "MUJER"')

for comuna, comuna_df in votos_mujeres.groupby('Comuna'):
    for elección_id, elección_df in comuna_df.groupby('ElecciónId'):
        total_mujeres = elección_df.query('Sexo == "MUJER"')['Votos Totales'].sum()
        total_hombres = elección_df.query('Sexo == "HOMBRE"')['Votos Totales'].sum()
        row = [comuna.lower(), elección_id, total_mujeres, total_hombres]
        data.append(row)
        
totales = pd.DataFrame(data, columns=['comuna', 'elección', 'mujeres', 'hombres'])
totales

In [ ]:
totales['ratio'] = totales.mujeres / totales.hombres

In [ ]:
totales.sort_values('ratio', ascending=False)[:10]

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
votos_mujeres.query('ElecciónId == "MUNICIPALES 2008"').query('Comuna == "RIO VERDE"').query('Sexo == "HOMBRE"')

In [ ]:
full_df['nombre_completo'] = full_df.Nombres.str.strip() + ' ' + full_df['Apellido Paterno'].str.strip() + ' ' + full_df['Apellido Materno'].str.strip()
full_df['nombre_completo'] = full_df.nombre_completo.str.strip()

In [ ]:
len(full_df.nombre_completo.unique())

In [52]:
data = []

for partido, partido_df in full_df.groupby('Partido'):
    total = len(partido_df.query('Sexo == "MUJER"').nombre_completo.unique())
    row = [partido, total]
    
    data.append(row)
    
candidatas = pd.DataFrame(data, columns=['partido', 'total'])

AttributeError: 'DataFrame' object has no attribute 'nombre_completo'

In [ ]:
partidos_dict = {
    'DEMOCRATA CRISTIANO': 'pdc',
    'PARTIDO DEMOCRATA CRISTIANO': 'pdc',
    
}

partidos_dict['DEMOCRATA CRISTIANO'], partidos_dict['PARTIDO DEMOCRATA CRISTIANO']

In [ ]:
candidatas.sort_values('total', ascending=False)[:10]

In [ ]:
geo_comunas['NOM_COM'] = geo_comunas.NOM_COM.str.lower()

In [ ]:
tres_comunas = ['las condes', 'vitacura', 'lo barnechea', 'providencia', 'la reina', 'ñuñoa', 'santiago']

totales.query('comuna in @tres_comunas').groupby('comuna').mean()

In [ ]:
geo_comunas.query('NOM_COM in @tres_comunas')

In [ ]:
totales['NOM_COM'] = totales.comuna
geo_totales = geo_comunas.query('NOM_COM in @tres_comunas').merge(totales.query('NOM_COM in @tres_comunas'), how='left', on='NOM_COM')
#geo_incidencia = geo_incidencia.to_crs({'init': 'epsg:3857'})
#ax = geo_incidencia.plot(column='Taza comuna', figsize=(16,20), alpha=0.4, edgecolor='k', cmap='Purples')

In [ ]:
import contextily as ctx

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

In [ ]:
from matplotlib import pyplot as plt


#Gráficos georeferenciados:
for elección, geo_data in geo_totales.groupby('elección'):
    geo_data = geo_data.to_crs({'init': 'epsg:3857'})
    ax = geo_data.plot(column='ratio', figsize=(16,20), alpha=0.4, edgecolor='k', cmap='Purples')
    add_basemap(ax, zoom=11)
    plt.title(elección)
    comunas_en_mapa = {
        'Lo Barnechea': (-7830000, -3935000),
    }
    for comuna, coordenadas in comunas_en_mapa.items():
        ax.annotate(comuna, coordenadas, xytext=coordenadas, fontsize=12)
        
    plt.savefig(f'Votos de mujeres a mujeres {elección}.png', bbox_inches='tight')

In [ ]:
def set_partido(row):
    if row['Candidato (a)'][:11] == 'VOTOS NULOS':
        row['Partido'] = 'Nulos'
    elif row['Candidato (a)'][:15] == 'VOTOS EN BLANCO':
        row['Partido'] = 'Blancos'
        
    return row

full_df = full_df.apply(set_partido, axis=1)

In [ ]:
full_df['Partido'] = full_df.Partido.str.strip()

In [ ]:
full_df['ElecciónId'] = full_df['Tipo de Elección'] + ' ' + full_df['Fecha de Elección'].astype(str).str[:4]

In [ ]:
results = []

for eleccion, local_df in full_df.query('Partido == "COMUNISTA DE CHILE"').groupby('id'):
    local_result = [eleccion, local_df['Fecha de Elección'].iloc[0].year, local_df['Votos Totales'].sum()]
    results.append(local_result)

resultados_pc = pd.DataFrame(results, columns=['nombre', 'año', 'total'])
resultados_pc

In [ ]:
from matplotlib import pyplot as plt


resultados_pc.set_index('año').sort_index().total.plot()

plt.title(f'resultados pc');

In [ ]:
full_df['election_kind'] = full_df['Tipo de Elección'] + ' ' + full_df['Votación Presidencial'].astype(str)

In [ ]:
full_df['election_kind'].unique()

In [ ]:
import seaborn as sns


for election_kind, sub_df in full_df.groupby('election_kind'):

    partidos = sub_df.Partido.value_counts()
    mask = partidos > 1000
    plot_data = partidos[mask]

    fig = plt.figure(figsize=(18, 18))

    sns.barplot(plot_data, plot_data.index, orient='h')

    plt.title(f'Votación partidos {election_kind}')
    
    plt.savefig(f'Votación partidos {election_kind}.png')

#plt.xticks(rotation=90)